In [1]:
%load_ext autoreload
%autoreload 2
from EventDriven.backtest import OptionSignalBacktest
from EventDriven.riskmanager.sizer import DefaultSizer
import pandas as pd
import numpy as np
from IPython.display import clear_output
from dbase.DataAPI.ThetaData import refresh_proxy_url, get_proxy_url
from copy import deepcopy
from helper import create_backtest_object


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-07-02 21:55:38 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://54.144.4.219:5500/thetadata


Scheduled Data Requests will be saved to: /Users/chiemelienwanisobi/cloned_repos/QuantTools/module_test/raw_code/DataManagers/scheduler/requests.jsonl


In [2]:
import os
os.environ['PROXY_URL'] =  ''
from dbase.DataAPI.ThetaData import refresh_proxy_url, get_proxy_url
get_proxy_url()

In [3]:
create_backtest_object

<function helper.create_backtest_object(cash, weights, trades_, max_cash)>

In [4]:
from EventDriven.riskmanager.sizer import ZscoreRVolSizer

In [5]:
import json
keys = [4, 5, 6, 7, 8, 9, 10, 11, 12 ]
# keys = [12]
imports_trades = {key: {'trades': pd.read_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_trades_{key}.csv').iloc[:, 1:]} for key in keys}

for key in keys:
    with open(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_weights_{key}.json', 'r') as f:
        weights = json.load(f)
        symbol_list = imports_trades[key]['trades'].Ticker.unique()
        untraded_symbols = [s for s in weights.keys() if s not in imports_trades[key]['trades'].Ticker.unique()]
        imports_trades[key]['weights'] = weights

In [50]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
filename = 'zscore2_dte180_roll120_smooth'
ending_capital = 20_000
initial_capital = ending_capital
capital_tracking = {}
objects = {}
for key in keys:
    print(f"Starting backtest for key: {key}, with initial capital: {ending_capital}")
    print(f"Previous Backtest Starting Capital: {initial_capital}, Ending capital from Previous Backtest: {ending_capital}, Return: {((ending_capital - initial_capital) / initial_capital) * 100}% \n")
    initial_capital = ending_capital
    print(f"Current Capital Tracking: {capital_tracking}")

    ## Set the trades and weights
    trades_ = imports_trades[key]['trades']
    weights = imports_trades[key]['weights']
    objects[key] = {}
    print(weights)

    ## Produce max cash map
    max_cash = {}
    cash = initial_capital
    for s, w in weights.items():
        print(f'{s} {w} {w * cash}')
        if w * cash > 500:
            max_cash[s] = 4
        elif w * cash > 300:
            max_cash[s] = 3
        elif w * cash > 200:
            max_cash[s] = 2
        elif w * cash > 100:
            max_cash[s] = 1
        else:
            max_cash[s] = 0.5
    print(max_cash)

    ## Create the backtest object
    evb_backtest = create_backtest_object(cash, weights, trades_, max_cash)
    evb_backtest.portfolio.roll_map = 120
    evb_backtest.portfolio.order_settings['target_dte'] = 270
    sizer = ZscoreRVolSizer(evb_backtest.portfolio, 
                            evb_backtest.risk_manager, 
                            evb_backtest.risk_manager.sizing_lev,
                            vol_type = 'weighted_mean')
    sizer.norm_constant = 2
    evb_backtest.portfolio.risk_manager.sizer = sizer
    
    # ## Structural Test
    # evb_backtest.portfolio.risk_manager.option_price = 'Midpoint_ewm_smooth'
    # evb_backtest.portfolio.risk_manager.submit_add_columns(('Midpoint', 'ewm_smooth'))

    evb_backtest.portfolio.risk_manager.print_settings()

    custom_attr = {
    'max_cash': max_cash,
    'w_map': evb_backtest.portfolio.weight_map,
    'initial_capital': initial_capital,
    'commission_rate': evb_backtest.executor.max_slippage_pct,
    'max_slippage_pct':evb_backtest.executor.commission_rate,
    'roll_map': evb_backtest.portfolio.roll_map,
    'order_settings': evb_backtest.portfolio.order_settings,
    'sizing_lev': evb_backtest.portfolio.risk_manager.sizing_lev,
    'limits': evb_backtest.portfolio.risk_manager.limits,
    't_plus_n': evb_backtest.portfolio.risk_manager.t_plus_n,
    'otm_moneyness_width': evb_backtest.portfolio.risk_manager.otm_moneyness_width,
    'itm_moneyness_width': evb_backtest.portfolio.risk_manager.itm_moneyness_width,
    }
    ## Run the backtest
    evb_backtest.run()

    ## Get the ending capital
    ending_capital = evb_backtest.portfolio._equity['Total'][-1]
    # initial_capital = ending_capital
    capital_tracking[key] = ending_capital
    objects[key]['WEIGHTS'] = evb_backtest.portfolio.weight_map
    objects[key]['END_CASH'] = ending_capital
    objects[key]['TEST_RESULT'] = {}
    objects[key]['TEST_RESULT']['BACKTESTER'] = evb_backtest
    objects[key]['TEST_RESULT']['AGG'] = evb_backtest.portfolio.aggregate()
    objects[key]['CUSTOM_ATTR'] = custom_attr
    clear_output(wait=True)

Starting backtest for key: 12, with initial capital: 20000
Previous Backtest Starting Capital: 20000, Ending capital from Previous Backtest: 20000, Return: 0.0% 

Current Capital Tracking: {}
{'NVDA': 0.3717903440438027, 'TSLA': 0.13285856444949615, 'META': 0.11763798538677593, 'NFLX': 0.08637009747959587, 'AMD': 0.0472536240089943, 'AAPL': 0.0676151517402857, 'COST': 0.0720078264760282, 'AMZN': 0.04879875274933317, 'BA': 0.02991981454758278, 'SBUX': 0.025747839118105225}
NVDA 0.3717903440438027 7435.806880876054
TSLA 0.13285856444949615 2657.171288989923
META 0.11763798538677593 2352.7597077355185
NFLX 0.08637009747959587 1727.4019495919174
AMD 0.0472536240089943 945.072480179886
AAPL 0.0676151517402857 1352.303034805714
COST 0.0720078264760282 1440.156529520564
AMZN 0.04879875274933317 975.9750549866635
BA 0.02991981454758278 598.3962909516556
SBUX 0.025747839118105225 514.9567823621045
{'NVDA': 4, 'TSLA': 4, 'META': 4, 'NFLX': 4, 'AMD': 4, 'AAPL': 4, 'COST': 4, 'AMZN': 4, 'BA': 4, '

In [44]:
# import dill

# with open(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/demos/opt_wfa/{filename}.pkl', 'wb') as f:
#     dill.dump(objects, f)

### RECREATE TEST:

Can we recreate exact backtest with parameters saved to excel

In [52]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    print(f'{s} {w} {w * cash}')
    if w * cash > 500:
        max_cash[s] = 4
    elif w * cash > 300:
        max_cash[s] = 3
    elif w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
print(max_cash)
recreate_evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash, t_plus_n=1, symbol_list = list(weights.keys()))
w_map = {x: w  * 0.95 for x, w in weights.items()}
recreate_evb_backtest.portfolio.weight_map = w_map
recreate_evb_backtest.portfolio.order_settings = {
                'type': 'spread',
                'specifics': [
                    {'direction': 'long', 'rel_strike': 1.0, 'dte': 365, 'moneyness_width': 0.1},
                    {'direction': 'short', 'rel_strike': 0.85, 'dte': 365, 'moneyness_width': 0.1} 
                ],
                'name': 'vertical_spread',
                'strategy': 'vertical',
                'target_dte': 270, #270
                'structure_direction': 'long',
                'spread_ticks': 1,
                'dte_tolerance': 60,
                'min_moneyness': 0.65, #0.75
                'max_moneyness': 1., #1.25
                'min_total_price': 0.95 #0.5
            }
recreate_evb_backtest.portfolio.max_contract_price = max_cash
recreate_evb_backtest.portfolio.roll_map = 120
recreate_evb_backtest.executor.commission_rate = 0.65/100
recreate_evb_backtest.portfolio.min_acceptable_dte_threshold = 95
recreate_evb_backtest.executor.max_slippage_pct = 0.075
recreate_evb_backtest.portfolio.risk_manager.sizing_lev = 4.5
recreate_evb_backtest.portfolio.risk_manager.max_slippage = 0.075
recreate_evb_backtest.portfolio.risk_manager.limits['delta'] = True
recreate_evb_backtest.portfolio.risk_manager.limits['dte'] = True
recreate_evb_backtest.portfolio.risk_manager.limits['moneyness'] = True
recreate_evb_backtest.portfolio.risk_manager.max_moneyness = 1.15
recreate_evb_backtest.portfolio.risk_manager.otm_moneyness_width = 0.45
recreate_evb_backtest.portfolio.risk_manager.itm_moneyness_width = 0.10
recreate_evb_backtest.portfolio.risk_manager.re_update_on_roll = False
recreate_evb_backtest.portfolio.risk_manager.max_tries = 15
sizer = ZscoreRVolSizer(recreate_evb_backtest.portfolio,
                        recreate_evb_backtest.risk_manager, 
                        recreate_evb_backtest.risk_manager.sizing_lev,
                        vol_type = 'weighted_mean')
sizer.norm_constant = 2
sizer.rolling_window = 100
sizer.rvol_window = (5, 20, 63)
sizer.weights = (0.5, 0.3, 0.2)
sizer.set_cash_rule(1)
recreate_evb_backtest.portfolio.risk_manager.sizer = sizer
recreate_evb_backtest.portfolio.risk_manager.print_settings()
recreate_evb_backtest.run()

NVDA 0.3717903440438027 7435.806880876054
TSLA 0.13285856444949615 2657.171288989923
META 0.11763798538677593 2352.7597077355185
NFLX 0.08637009747959587 1727.4019495919174
AMD 0.0472536240089943 945.072480179886
AAPL 0.0676151517402857 1352.303034805714
COST 0.0720078264760282 1440.156529520564
AMZN 0.04879875274933317 975.9750549866635
BA 0.02991981454758278 598.3962909516556
SBUX 0.025747839118105225 514.9567823621045
{'NVDA': 4, 'TSLA': 4, 'META': 4, 'NFLX': 4, 'AMD': 4, 'AAPL': 4, 'COST': 4, 'AMZN': 4, 'BA': 4, 'SBUX': 4}

Risk Manager Settings:
Start Date: 2025-01-03 00:00:00
End Date: 2025-03-03 00:00:00
Current Limits State (Position Adjusted when these thresholds are reached):
    Delta: True
    Gamma: False
    Vega: False
    Theta: False
    Roll On DTE: True
        Min DTE Threshold: 95
    Roll On Moneyness: True
        Max Moneyness: 1.15
Quanitity Sizing Type: delta
            
No positions need to be adjusted on 2025-01-03 00:00:00
Risk Manager Actions:  NO_POSITIO

In [53]:
re_equity = recreate_evb_backtest.portfolio._equity
(re_equity.iloc[-1,:]/ re_equity.iloc[0,:] )-1

NVDA          0.000000
TSLA         -0.524514
META          0.516175
NFLX          0.438471
AMD           0.000000
AAPL         -0.397570
COST          1.130964
AMZN         -0.606141
BA           -0.147614
SBUX          0.728486
cash          0.000000
commission         inf
Total         0.064813
dtype: float64

In [46]:
recreate_evb_backtest.portfolio.risk_manager.sizer.cash_rule
recreate_evb_backtest.portfolio.allocated_cash_map

{'NVDA': 7064.0165368322505,
 'TSLA': 70.096556356988,
 'META': 3039.4589978751073,
 'NFLX': 2258.762330316351,
 'AMD': 897.8188561708916,
 'AAPL': 663.0426011988824,
 'COST': 2842.6582059200227,
 'AMZN': 314.43391488016704,
 'BA': 425.2707183363563,
 'SBUX': 842.6754165475909}